<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Neural_Architecture_Search_(NAS).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install autokeras

In [ ]:
pip install tensorflow

In [ ]:
import autokeras as ak
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess the data
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1)) / 255.0  # Normalize and add channel dimension
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1)) / 255.0
y_train = to_categorical(y_train, 10)  # One-hot encode labels
y_test = to_categorical(y_test, 10)

# Define the AutoKeras ImageClassifier
clf = ak.ImageClassifier(max_trials=10, overwrite=True)  # Try up to 10 different architectures

# Train the model
clf.fit(x_train, y_train, epochs=10)

# Evaluate the model
accuracy = clf.evaluate(x_test, y_test)
print(f"Test accuracy: {accuracy}")

# Export the best model found
model = clf.export_model()

# Save the model for later use
model.save("best_image_classifier.h5")
print("Best model saved as best_image_classifier.h5")